# AI Data Analysis
**Carson Case: carsonpcase@gmail.com**

### Instructions in README

In [8]:
# Set up env variables
import os
ASTRA_DB_BUNDLE_PATH =os.getenv("ASTRA_DB_BUNDLE_PATH")
ASTRA_DB_TOKEN = os.getenv("TOKEN")
ASTRA_DB_CLIENT_ID = os.getenv("CLIENT_ID")
ASTRA_DB_CLIENT_SECRET = os.getenv("SECRET")
ASTRA_DB_KEYSPACE = os.getenv("KEYSPACE")
OPENAI_KEY= os.getenv("OPENAI_KEY")

In [9]:
# Import libraries
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain_core.messages import HumanMessage, SystemMessage

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

from datasets import load_dataset, Dataset

In [11]:
# Config Astra
cloud_config = {
    "secure_connect_bundle":  ASTRA_DB_BUNDLE_PATH
}
auth_provider = PlainTextAuthProvider(ASTRA_DB_CLIENT_ID, ASTRA_DB_CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
astra_session = cluster.connect()

chat = ChatOpenAI(openai_api_key=OPENAI_KEY, model="gpt-3.5-turbo-16k", max_tokens=500)
myEmbedding = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)

In [12]:
# Create Cassandra Store and table if it doesn't exist
listingCassandraStore = Cassandra(
    embedding=myEmbedding,
    session=astra_session,
    keyspace=ASTRA_DB_KEYSPACE,
    table_name="listings"
)

In [22]:
vectorIndex = VectorStoreIndexWrapper(vectorstore=listingCassandraStore)

query = "I have a 2018 Ford F150 for $20,000 for sale"

data = listingCassandraStore.similarity_search(query, k=6)
messages = [
    SystemMessage(content="You are looking to buy a used car. You are a very smart and well educated person who knows the current market data for used cars. Provide an educated analysis of offers presented to you. Explicitly say at the end of your message if you are interested or not interested in the offer. The following is the relevant market data at your disposal: "+str(data)),
    HumanMessage(content=query),
]

chat.invoke(messages)


AIMessage(content='Based on the market data provided, the average price for a 2018 Ford F150 is around $26,995. Your offer of $20,000 is significantly lower than the market average. It would be advisable to consider adjusting your price to be more in line with the market value in order to attract potential buyers.')